# 손글씨 숫자 이미지 분류: 다양한 모델 성능 비교

이 노트북은 `scikit-learn`에 내장된 **손글씨 숫자(digits) 데이터셋**을 사용하여, 0부터 9까지의 숫자를 분류하는 다중 클래스 분류 문제를 해결합니다.

로지스틱 회귀, KNN, 의사결정나무, 랜덤 포레스트, 그레이디언트 부스팅 등 다양한 분류 알고리즘을 사용하여 모델을 학습시키고, 각각의 성능을 비교 분석하는 것을 목표로 합니다.

### 1. 라이브러리 임포트

In [ ]:
# 데이터 로드 및 시각화
import matplotlib.pyplot as plt
from sklearn.datasets import load_digits
import numpy as np

# 데이터 분할
from sklearn.model_selection import train_test_split

# 분류 모델들
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

### 2. 데이터 준비 및 탐색

`load_digits()` 함수로 데이터를 불러옵니다. 이 데이터셋의 각 이미지는 8x8 픽셀 크기의 흑백 이미지입니다.
- `data.data` (또는 `X`): 각 8x8 이미지를 64개의 픽셀 값으로 펼친 1차원 배열. 모델의 입력으로 사용됩니다.
- `data.images`: 원본 8x8 형태를 유지하고 있는 이미지 데이터. 시각화에 사용됩니다.
- `data.target` (또는 `y`): 각 이미지에 해당하는 실제 숫자(0~9). 모델이 맞춰야 할 정답입니다.

In [ ]:
# 데이터 로드
digits = load_digits()
X = digits.data
y = digits.target
images = digits.images

print("입력 데이터(X) 형태:", X.shape) # (샘플 수, 특성 수) -> (1797, 64)
print("원본 이미지 데이터 형태:", images.shape) # (샘플 수, 세로 픽셀, 가로 픽셀) -> (1797, 8, 8)
print("타겟 데이터(y) 형태:", y.shape)

### 3. 데이터 시각화

데이터가 어떻게 생겼는지 직접 눈으로 확인하는 것은 중요합니다. 처음 10개의 이미지를 라벨과 함께 출력해 보겠습니다.

In [ ]:
def draw_digits():
    plt.figure(figsize=(10, 5))
    plt.suptitle("Handwritten Digits Sample", fontsize=16, y=1.02)
    
    for i in range(10):
        # 2행 5열의 그리드에서 i+1번째 위치에 그림을 그립니다.
        plt.subplot(2, 5, i + 1)
        
        # 이미지를 흑백(gray_r)으로 표시합니다.
        plt.imshow(images[i], cmap="gray_r")
        
        # 이미지의 실제 라벨을 제목으로 표시합니다.
        plt.title(f"Label: {y[i]}")
        
        # 축 정보를 숨깁니다.
        plt.axis('off')
        
    plt.tight_layout() # 서브플롯 간의 간격을 자동으로 조절합니다.
    plt.show()

draw_digits()

### 4. 훈련 및 테스트 데이터 분할

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, test_size=0.3)

print(f"훈련 데이터 형태: {X_train.shape}")
print(f"테스트 데이터 형태: {X_test.shape}")

### 5. 다양한 분류 모델 학습 및 성능 비교

이제 준비된 데이터로 여러 분류 모델을 학습시키고, 각 모델의 정확도를 비교해 보겠습니다.

In [ ]:
# 결과를 저장할 딕셔너리
results = {}

def train_and_evaluate(model_name, model):
    """주어진 모델을 학습하고 평가 결과를 출력 및 저장합니다."""
    model.fit(X_train, y_train)
    train_score = model.score(X_train, y_train)
    test_score = model.score(X_test, y_test)
    results[model_name] = test_score
    print(f"--- {model_name} ---")
    print(f"훈련 세트 정확도: {train_score:.4f}")
    print(f"테스트 세트 정확도: {test_score:.4f}")
    print("-"*30)

In [ ]:
# 1. 로지스틱 회귀
lr_model = LogisticRegression(solver='liblinear', multi_class='auto', max_iter=5000, random_state=0)
train_and_evaluate("Logistic Regression", lr_model)

# 2. K-최근접 이웃
knn_model = KNeighborsClassifier(n_neighbors=3)
train_and_evaluate("K-Nearest Neighbors", knn_model)

# 3. 의사결정나무
dt_model = DecisionTreeClassifier(max_depth=5, random_state=0)
train_and_evaluate("Decision Tree", dt_model)

# 4. 랜덤 포레스트
rf_model = RandomForestClassifier(max_depth=4, n_estimators=100, random_state=0)
train_and_evaluate("Random Forest", rf_model)

# 5. 그레이디언트 부스팅
gb_model = GradientBoostingClassifier(max_depth=4, n_estimators=100, random_state=0, learning_rate=0.1)
train_and_evaluate("Gradient Boosting", gb_model)

### 6. 결과 요약

모든 모델의 테스트 세트 정확도를 시각화하여 한눈에 비교해 보겠습니다.

In [ ]:
# 결과 딕셔너리를 DataFrame으로 변환하여 정렬
results_df = pd.DataFrame.from_dict(results, orient='index', columns=['Test Accuracy'])
results_df = results_df.sort_values(by='Test Accuracy', ascending=False)

print(results_df)

# 결과 시각화
plt.figure(figsize=(10, 6))
results_df['Test Accuracy'].plot(kind='barh', color='skyblue')
plt.title("Model Performance Comparison")
plt.xlabel("Test Set Accuracy")
plt.xlim(0.9, 1.0) # x축 범위 조절
plt.grid(axis='x', linestyle='--')
plt.show()
# 이 데이터셋에서는 KNN 모델이 가장 좋은 성능을 보였습니다.